# E30 - Lincolnshire Poacher numbers station simulator


Messages are normalized to Crockford base32 (`0123456789ABCDEFGHJKMNPQRSTVWXYZ`)
Groups of 3 letters are combined to make a 15 bit group
Bits are xored with a one time pad
15 bit groups (total of 32768 possible combinations) are mapped to a 5 digit group (100000 combinations) using a multiplier and modulus

Messages are sent in this format:

```
P = "The Lincolnshire Poacher song"
C = "chime"


P P `pad_num` C `encrypted number groups` C P
```

https://en.wikipedia.org/wiki/Lincolnshire_Poacher_(numbers_station)



Start with initializing some common data and load the provided one time pad book (you should generate your own, see below)

In [ ]:
from textwrap import wrap
import os
import sys
import time
import base32_crockford

multiplier = 165321

group_table_send = {}
group_table_receive = {}

for num in range(0, 2**15):
    group_table_send.update({format(num, '015b'): format(divmod(num * multiplier, 100000)[1], '05')})
    group_table_receive.update({format(divmod(num * multiplier, 100000)[1], '05'): format(num, '015b')})

with open("pad.txt", "r") as f:
    pads = f.readlines()    

Run this next bit to generate your very own one time pad book

In [ ]:
'''generate one time pad book'''
import random
with open("pad.txt", "w+") as f:
    for pad_num in range(0, 100000):
        pad = ""
        for _ in range(0,256):

            pad += base32_crockford.encode(format(random.randint(0,31)))
        f.write(pad + "\n")

Select a `pad_num` and put your secret message in `plaintext`

In [ ]:
'''encoder'''

pad_num = 23453
plaintext = "what even is a plate of beets"

pad = list(pads[pad_num]) 

plaintext_sanitized = plaintext.replace("u", "v").replace(" ", "")
plaintext_normalized = base32_crockford.normalize(plaintext_sanitized)

while divmod(len(plaintext_normalized), 3)[1]:
    plaintext_normalized += "0"

print(plaintext_normalized)

letter_groups = wrap(plaintext_normalized, 3)

encoded_groups = []

for group in letter_groups:
    binary_groups = ""
    for letter in group:
        decrypted_int_letter = base32_crockford.decode(letter)
        encrypted_int_letter = decrypted_int_letter ^ base32_crockford.decode(pad.pop(0))
        binary_groups += format(encrypted_int_letter, '05b')
    encoded_groups += [group_table_send[binary_groups]]

#print(encoded_groups)

def say_group(group):
    group_len = len(group)
    group = enumerate(group)
    
    for num in group:
        print(num[1], end='')
        if num[0] == group_len - 1:
            os.system(f"afplay {num[1]}final.wav")
        else:
            os.system(f"afplay {num[1]}.wav")
    print(" ", end='')
    time.sleep(1)
    
    
for _ in range(1,3):
    print("P ", end='')  
    # Audio("./p.wav", autoplay=True)
    os.system("afplay p.wav")
  

say_group(str(pad_num))
    
for _ in range(1,2):
    print("C ", end='')
    os.system("afplay c.wav")

for group in encoded_groups:
    say_group(group)

    
for _ in range(1,2):
    print("C ", end='')
    os.system("afplay C.wav")

    
for _ in range(1,2):
    print("P ", end='')
    os.system("afplay p.wav")



After transcribing a message, change the `pad_num` to the transcribed one and put your formatted number groups in `ciphertext`.

In [ ]:
'''decoder'''

pad_num = 23453
ciphertext = "51801 68482 96325 71979 62822 04564 21967 94292"

pad = list(pads[pad_num])

encoded_groups = ciphertext.split()

for group in encoded_groups:
    binary_groups = wrap(group_table_receive[group], 5)
    for binary in binary_groups:
        encrypted_int_letter = int(binary, 2)
        decrypted_int_letter = encrypted_int_letter ^ base32_crockford.decode(pad.pop(0))
        letter = base32_crockford.encode(str(decrypted_int_letter))
        print(letter, end='')